In [ ]:

import datetime
import pandas as pd
import matplotlib as plt
import numpy as np
import os
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic
from hdbscan import HDBSCAN
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 50)
print(os.getcwd())

In [ ]:
import pickle

In [ ]:
objects = []
with (open("Tweet_16404141.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [ ]:
import os
import pandas as pd

def collect_csv_files(directory):
    csv_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

def combine_csv_files(csv_files):
    combined_df = pd.DataFrame()
    for file in csv_files:
        try:
            df = pd.read_csv(file)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        except pd.errors.EmptyDataError:
            continue
        except Exception as e:
            continue
    return combined_df

directory = '/disk/mnemo/users/randomuser/SBERT All Embedding/Group Tweets/'
csv_files = collect_csv_files(directory)
combined_df = combine_csv_files(csv_files)



In [ ]:
combined_df.to_csv('All_Recent_Posts.csv', index = False)

In [ ]:
len(combined_df)

In [ ]:
combined_df_unique = combined_df.drop_duplicates('Tweet_ID').reset_index(drop = True)

In [ ]:
combined_df_unique

In [ ]:
combined_df_unique.columns

In [ ]:
combined_df_unique.text[12344][0:2] == 'RT'

In [ ]:
import pandas as pd


filtered_df = combined_df_unique[combined_df_unique['text'].str.startswith('RT')]


In [ ]:
combined_df_unique_tweet = combined_df_unique[~combined_df_unique.Tweet_ID.isin(filtered_df.Tweet_ID.tolist())]

In [ ]:
len(combined_df_unique_tweet)

In [ ]:
combined_df_unique.groupby('User_ID').count().loc[combined_df_unique.groupby('User_ID').count().Tweet_ID > 20]

In [ ]:
combined_df_unique

In [ ]:
combined_df_unique['Date'] = pd.to_datetime(combined_df_unique['created_at'], errors='coerce')
combined_df_unique['Only_Date'] = combined_df_unique['Date'].dt.date

In [ ]:
combined_df_unique['Only_Date'] = combined_df_unique['Date'].dt.date

In [ ]:
combined_df_unique.loc[combined_df_unique.Only_Date >= datetime.date(2024, 4, 19)].groupby('User_ID').count()

In [ ]:
combined_df_unique

In [ ]:
combined_df_unique

In [ ]:
combined_df_unique_tweet = combined_df_unique_tweet.reset_index(drop= True)

In [ ]:
df_recent_post_cluster = pd.read_csv('Recent_Post_Tweet_Cluster.csv')

In [ ]:
df_recent_post_cluster = pd.merge(combined_df_unique_tweet, df_recent_post_cluster)

In [ ]:
df_recent_post_cluster

In [ ]:
df_remove_spam = df_recent_post_cluster.drop_duplicates(['User_ID', 'Cluster_ID']).groupby('Cluster_ID').count().reset_index()

In [ ]:
df_remove_spam[df_remove_spam.Tweet_ID >= 2].Cluster_ID.tolist()

In [ ]:
df_recent_post_cluster[df_recent_post_cluster.Cluster_ID.isin(df_remove_spam[df_remove_spam.Tweet_ID >= 2].Cluster_ID.tolist())].groupby("Cluster_ID").count()

In [ ]:
df_final_cluster_new = df_recent_post_cluster[df_recent_post_cluster.Cluster_ID.isin(df_remove_spam[df_remove_spam.Tweet_ID >= 2].Cluster_ID.tolist())].reset_index(drop = True)

In [ ]:
df_final_cluster_new['Cluster_ID'] = 'New_Wild_' + df_final_cluster_new['Cluster_ID'].astype('str')

In [ ]:
df_final_cluster_new.to_csv('Recent_Post_Cluster_Final_Updated.csv', index = False)

In [ ]:
df_final_cluster_new.groupby('Cluster_ID').count()

In [ ]:
df_group = pd.read_csv('Refined_Community_with_Political_Label.csv')

In [ ]:
df_group

In [ ]:
df_final_cluster_new = pd.merge(df_final_cluster_new, df_group, on = 'User_ID')

In [ ]:
df_final_cluster_new = df_final_cluster_new.drop(columns = 'index')

In [ ]:
df_final_cluster_new['Date'] =  pd.to_datetime(df_final_cluster_new['created_at'], errors='coerce')

In [ ]:
df_final_cluster_new['Only_Date'] = df_final_cluster_new['Date'].dt.date

In [ ]:
df_final_cluster_new.columns

In [ ]:
df_final_cluster_new.sort_values(['Cluster_ID', 'Date'])

In [ ]:
df_final_cluster_new.columns

In [ ]:
df_final_cluster_new

In [ ]:
df_final_cluster_new[df_final_cluster_new.Cluster_ID == 'New_Wild_124']

In [ ]:
df_final_cluster_new = df_final_cluster_new[~df_final_cluster_new.Cluster_ID.isin(['New_Wild_123', 'New_Wild_0'])]

In [ ]:
df_final_cluster_new.to_csv('Recent_Post_Cluster_Final2.csv', index = False)

In [ ]:
df_final_cluster_new.groupby('Cluster_ID').count().sort_values('User_ID')

In [ ]:
df_final_cluster_new[df_final_cluster_new.Cluster_ID == 'New_Wild_3'].groupby('Community_ID').count()

In [ ]:
df_final_cluster_new2.columns

In [ ]:
bjp_group = [0,8,1,6,15,13,5,14,4]
inc_group = [17,18,19,42,29]
top_group = bjp_group + inc_group

In [ ]:
df_final_cluster_new2 = df_final_cluster_new[df_final_cluster_new.Community_ID.isin(top_group)]

In [ ]:
df_count_cluster = df_final_cluster_new2.groupby('Cluster_ID').count().reset_index()

In [ ]:
df_count_cluster = df_count_cluster[['Cluster_ID', 'Tweet_ID']].rename(columns = {'Tweet_ID': 'Total_Post'})

In [ ]:
df_count_cluster

In [ ]:
cluster_user_count = df_final_cluster_new2.groupby(['Cluster_ID', 'Community_ID'])['User_ID'].nunique().reset_index(name='Cluster_User_Count')

In [ ]:
total_users_per_cluster = df_final_cluster_new2.groupby('Cluster_ID')['User_ID'].nunique().reset_index(name='Total_Users')

In [ ]:
merged_cluster_data = pd.merge(cluster_user_count, total_users_per_cluster, on='Cluster_ID')

In [ ]:
merged_cluster_data['Percentage'] = (merged_cluster_data['Cluster_User_Count']/merged_cluster_data['Total_Users']) * 100

In [ ]:
merge_controlled = merged_cluster_data.loc[merged_cluster_data.Percentage>50]

In [ ]:
merge_controlled[merge_controlled.Community_ID.isin(bjp_group)]

In [ ]:
merge_controlled[merge_controlled.Community_ID.isin(inc_group)]

In [ ]:
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic
from hdbscan import HDBSCAN

In [ ]:
merge_controlled.Cluster_ID

In [ ]:
df_cluster_controlled = df_final_cluster_new2[df_final_cluster_new2.Cluster_ID.isin(merge_controlled.Cluster_ID)]

In [ ]:
df_cluster_controlled_topic = df_final_cluster_new2[df_final_cluster_new2.Cluster_ID.isin(merge_controlled.Cluster_ID)].drop_duplicates("Cluster_ID")

In [ ]:
df_cluster_controlled_topic.columns

In [ ]:
import re
def remove_twitter_mentions(text):
    return re.sub(r'@\w+', '', text)

def remove_links(text):
    return re.sub(r'http\S+', '', text)


df_cluster_controlled_topic['Refined_Text'] = df_cluster_controlled_topic['text'].apply(remove_twitter_mentions)
df_cluster_controlled_topic['Refined_Text'] = df_cluster_controlled_topic['Refined_Text'].apply(remove_links)
df_cluster_controlled_topic['Refined_Text'] = df_cluster_controlled_topic['Refined_Text'].str.replace('\n', ' ')
df_cluster_controlled_topic['Refined_Text'] = df_cluster_controlled_topic['Refined_Text'].str.strip()

In [ ]:
doc_all = df_cluster_controlled_topic.Refined_Text.tolist()

In [ ]:
from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance
from sklearn.datasets import fetch_20newsgroups
from bertopic.representation import ZeroShotClassification


main_representation = KeyBERTInspired()
hdbscan_model = HDBSCAN(min_cluster_size=2, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=2)


aspect_model1 = PartOfSpeech("en_core_web_sm")
aspect_model2 = [KeyBERTInspired(top_n_words=30), MaximalMarginalRelevance(diversity=.5)]

representation_model = {
   "Main": main_representation,
   "Aspect1":  aspect_model1,
   "Aspect2":  aspect_model2 
}


In [ ]:
topic_model_all = BERTopic(representation_model=representation_model, language="multilingual", hdbscan_model=hdbscan_model)
topics_found, probs_found = topic_model_all.fit_transform(doc_all)

In [ ]:
topic_model_all.save("/disk/mnemo/users/randomuser/SBERT All Embedding/Group_Topic/New_Tweet_All_Topic", serialization="pickle")

In [ ]:
new_topics = topic_model_all.reduce_outliers(doc_all, topics_found)

In [ ]:
topic_model_all.get_topic_info()

In [ ]:
new_topics

In [ ]:
df_cluster_controlled_topic['Actual_Topic'] = topics_found
df_cluster_controlled_topic['Reduced_Topic'] = new_topics

In [ ]:
df_cluster_controlled_topic.groupby('Reduced_Topic').count()

In [ ]:
df_cluster_controlled_topic= pd.read_csv('New_Post_with_Updated_Topics.csv')

In [ ]:
bjp_group = [0,8,1,6,15,13,5,14,4]
inc_group = [17,18,19,42,29]
top_group = bjp_group + inc_group

In [ ]:
df_group = pd.read_csv('Refined_Community_with_Political_Label.csv')

In [ ]:
df_group_top = df_group[df_group.Community_ID.isin(top_group)]

In [ ]:
df_cluster_controlled_topic = df_cluster_controlled_topic[df_cluster_controlled_topic.User_ID.isin(df_group_top.User_ID)]

In [ ]:
df_cluster_controlled_topic.columns

In [ ]:
df_topic_top = df_cluster_controlled_topic

In [ ]:
df_topic_top_count = df_topic_top.groupby(['Reduced_Topic', 'Community_ID']).count()
df_topic_top_count2 = df_topic_top_count.reset_index()

In [ ]:
topic_group = {}
for i in range(0, 333):
    topic_group[i] = df_topic_top_count2.loc[df_topic_top_count2.Reduced_Topic == i].Community_ID.tolist()

In [ ]:
bjp_control = 0
inc_control = 0
for i, j in topic_group.items():
    len_item = len(j)
    len_intersect_bjp = len(set(j).intersection(bjp_group))
    len_intersect_bjp = len(set(j).intersection(inc_group))
    if len_item == len_intersect_bjp:
        bjp_control += 1
    if len_item == len_intersect_inc:
        inc_control += 1

In [ ]:
inc_control

In [ ]:
topic_group

In [ ]:
dict_group = {}
for key,values in topic_group.items():
    for i in values:
        if i not in dict_group.keys():
            dict_group[i] = 1
        else:
            dict_group[i] = dict_group[i] + 1

In [ ]:
sum(dict_group.values())

In [ ]:
265+299

In [ ]:
print(bjp_control)

In [ ]:
print(inc_control)

In [ ]:
len_intersect_inc

In [ ]:
len_intersect_bjp

In [ ]:
topic_group

In [ ]:
df_topic_top

In [ ]:
topic_model_all.get_topic_info().iloc[10:20]

In [ ]:
df_topic_top[df_topic_top.Reduced_Topic == 15].Cluster_ID

In [ ]:
similar_topics, similarity = topic_model_all.find_topics("china", top_n=5)

In [ ]:
similar_topics

In [ ]:
df_topic_top[df_topic_top.Reduced_Topic == 189].Cluster_ID.tolist()

In [ ]:
df_topic_top[df_topic_top.Reduced_Topic == 15].groupby('Influence_ID_Update').count()

In [ ]:
df_topic_top[df_topic_top.Reduced_Topic == 15]

In [ ]:
df_topic_top.columns

In [ ]:
New_Wild_213

In [ ]:
df_final_cluster_new2.loc[df_final_cluster_new2.Cluster_ID.isin(df_topic_top[df_topic_top.Reduced_Topic == 189].Cluster_ID.tolist())]

In [ ]:
df_final_cluster_new2.groupby('Cluster_ID').count()

In [ ]:
df_final_cluster_new.columns

In [ ]:
df_final_cluster_new['Total_Engagement'] = df_final_cluster_new.favorite_count + df_final_cluster_new.retweet_count

In [ ]:
df_final_cluster_new[df_final_cluster_new.Cluster_ID.isin(df_topic_top[df_topic_top.Reduced_Topic == 15].Cluster_ID)].Total_Engagement.sum()

In [ ]:
from datetime import datetime
def convert_to_date(datetime_str):
    return datetime.strptime(datetime_str, '%a %b %d %H:%M:%S %z %Y').date()

df_final_cluster_new['Date'] = df_final_cluster_new['created_at'].apply(convert_to_date)

In [ ]:
df_final_cluster_new[df_final_cluster_new.Cluster_ID.isin(df_topic_top[df_topic_top.Reduced_Topic == 15].Cluster_ID)].groupby(['Party_Label','Date']).count()

In [ ]:
df_topic_top[df_topic_top.Reduced_Topic == 15].columns

In [ ]:
df_final_cluster_new[['Cluster_ID','text']].sample(20)

In [ ]:
df_final_cluster_new[df_final_cluster_new.Cluster_ID.isin(df_topic_top[df_topic_top.Reduced_Topic == 15].Cluster_ID)][['Cluster_ID', 'text']].iloc[20:40]

In [ ]:
df_topic_top.Community_ID

In [ ]:
bjp_group = [0,8,1,6,15,13,5,14,4]
inc_group = [17,18,19,42,29]
top_group = bjp_group + inc_group

In [ ]:
df_topic_top['Influence_ID'] = '-1'

In [ ]:
for i, j in topic_group.items():
    for k in j:
        if k in bjp_group:
            df_topic_top.loc[((df_topic_top.Reduced_Topic == i) & (df_topic_top.Community_ID== k)), 'Influence_ID'] = 'BJP_Influence_' + str(i)
        elif k in inc_group:
            df_topic_top.loc[((df_topic_top.Reduced_Topic == i) & (df_topic_top.Community_ID== k)), 'Influence_ID'] = 'INC_Influence_' + str(i)

In [ ]:
df_topic_top.groupby(['Influence_ID']).count()

In [ ]:
df_topic_top['Influence_ID_Update'] = df_topic_top['Influence_ID'] + '_' + df_topic_top.Community_ID.astype('str')

In [ ]:
df_topic_top.groupby(['Influence_ID_Update']).count().reset_index().

In [ ]:
df_topic_top[df_topic_top.Community_ID.isin(bjp_group)].groupby('Influence_ID_Update').count()

In [ ]:
df_topic_top.columns

In [ ]:
topic_group

In [ ]:
df_topic_top[(df_topic_top.Reduced_Topic == 1) & (df_topic_top.Community_ID== 1)]